In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df_infos = pd.read_csv('../saved/infos.csv')
fDataReNorms = np.load('../saved/fDataReNorms.npy')

In [4]:
#df_infos

In [5]:
# from sklearn.preprocessing import normalize
# #fDataReNorms=[fDataRe-np.tile(fDataRe[:,0],(250,1)).transpose() for fDataRe in fDataRes]
# fDataReNorms=np.array([normalize(fDataRe) for fDataRe in fDataRes])
# np.save('../saved/fDataReNorms.npy', fDataReNorms)
# [plt.plot(fDataReNorm[6]) for fDataReNorm in fDataReNorms]

In [6]:
# fDataReNormAFs = fDataReNorms[df_infos['AF']]

# for fDataReNormAF in fDataReNormAFs[:10]:
#     plt.plot(fDataReNormAF[0])
#     plt.show()

In [7]:
from collections import Counter
from xgb.global_vars import headers, labels
print(Counter(np.sum(df_infos[labels], axis=1)))

Counter({1: 100528})


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [9]:
y = torch.FloatTensor(df_infos[labels].to_numpy()).to(device)

In [10]:
X = torch.FloatTensor(fDataReNorms).to(device)

In [11]:

class SimpleNet(nn.Module):
    def __init__(self, n_input, n_h1, n_h2, n_h3, verbose=False):
        super(SimpleNet, self).__init__()
        self.fc1        = nn.Linear(n_input, n_h1)
        self.fc2        = nn.Linear(n_h1, n_h2)
        self.fc3        = nn.Linear(120, n_h3)
        self.fc4        = nn.Linear(n_h3, 9)
        self.dropout = nn.Dropout(p=0.5)
        self.verbose=verbose

    def forward(self, x):
        if self.verbose:
            print('0: ', x.shape)
        x = self.fc1( x )     
        x = self.dropout(x)
        if self.verbose:
            print('1: ', x.shape)
        x = F.relu(   x )     
        x = self.fc2( x )    
        if self.verbose:
            print('2: ', x.shape)
        x = F.relu(   x )      
        if self.verbose:
            print('3: ', x.shape)
        x = x.view(x.shape[0], -1)
        x = self.fc3( x )  
        if self.verbose:
            print('4: ', x.shape)
        x = F.relu(   x )   
        x = self.fc4(x)
        if self.verbose:
            print('5: ', x.shape)
        return x

model = SimpleNet(250, 250, 10, 10)
model.forward(X.cpu())

tensor([[ 0.1460, -0.0918, -0.0784,  ..., -0.1310,  0.1038,  0.2622],
        [ 0.1455, -0.0913, -0.0802,  ..., -0.1349,  0.0945,  0.2687],
        [ 0.1466, -0.0911, -0.0770,  ..., -0.1378,  0.1002,  0.2589],
        ...,
        [ 0.1508, -0.0918, -0.0766,  ..., -0.1359,  0.1022,  0.2576],
        [ 0.1479, -0.0942, -0.0788,  ..., -0.1290,  0.1080,  0.2608],
        [ 0.1478, -0.0902, -0.0768,  ..., -0.1330,  0.0995,  0.2624]],
       grad_fn=<AddmmBackward>)

In [ ]:
#%%
from sklearn.metrics import accuracy_score
from evaluate_12ECG_score import compute_beta_score, compute_auc

def evaluate_beta(output, y):
    
    accuracy,f_measure,f_beta,g_beta = compute_beta_score(labels=y, 
                       output=output, 
                       beta=2, num_classes=1)
    
    auroc, auprc = compute_auc(labels=y, 
                                probabilities=output,
                                num_classes=1)

    return accuracy,f_measure,f_beta,g_beta, auroc, auprc


def binary_acc(y_preds, y_tests):
    accs = []
    for i in range(9):
        y_pred, y_test = y_preds[:,i], y_tests[:,i]
        
        y_pred_tag = torch.round(torch.sigmoid(y_pred))

        correct_results_sum = (y_pred_tag == y_test).sum().float()
        acc = correct_results_sum/y_test.shape[0]
        #acc = torch.round(acc * 100)
        #print(acc)
        accs.append(acc.data.cpu().numpy())
    return accs

from snippets.pytorchtools import EarlyStopping
from sklearn.model_selection import KFold
import time

st = time.time()
patience = 200
kf = KFold(10)

saved_dir = '../saved/model/'
for i, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    model = SimpleNet(250, 250, 10, 10).to(device)
    learning_rate = 0.01
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    criterion = nn.BCEWithLogitsLoss()

    losses_train = []
    losses_test = []

    avg_losses_train = []
    avg_losses_test = []


    early_stopping = EarlyStopping(patience, verbose=False, 
                                saved_dir=saved_dir, save_name=str(i))

    for epoch in range(2000):
        train_loss = 0
        validation_loss = 0

        optimizer.zero_grad()
        output_train = model(X[train_idx])
        output_test = model(X[test_idx])

        loss_train = criterion(output_train, y[train_idx])
        loss_test = criterion(output_test, y[test_idx])

        loss_train.backward()
        optimizer.step()

        losses_train.append(loss_train.item())
        losses_test.append(loss_test.item())

        avg_loss_train = np.average(losses_train)
        avg_loss_test = np.average(losses_test)

        avg_losses_train.append(avg_loss_train)
        avg_losses_test.append(avg_loss_test)

        if epoch % 50 == 0:
            accuracy = binary_acc(output_train, y[train_idx])
            accuracy2 = binary_acc(output_test, y[test_idx])
            accuracy_p = ' '.join(['{:.2f},'.format(acc)for acc in accuracy])
            accuracy2_p = ' '.join(['{:.2f},'.format(acc)for acc in accuracy2])
            print('S{}/{} {:.2f} min|\nTrain Loss: {:.6f}, Acc: {}|\nValid Loss: {:.6f}, Acc: {}|\n '.format(
                i, epoch, (time.time()-st)/60,
                avg_loss_train, accuracy_p, avg_loss_test, accuracy2_p))

        early_stopping(avg_loss_test, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break


S0/0 0.00 min|
Train Loss: 0.642301, Acc: 0.82, 0.91, 0.03, 0.86, 0.90, 0.93, 0.77, 0.88, 0.97,|
Valid Loss: 0.641723, Acc: 0.82, 0.91, 0.04, 0.85, 0.89, 0.93, 0.78, 0.89, 0.98,|
 
S0/50 0.19 min|
Train Loss: 0.351661, Acc: 0.82, 0.91, 0.97, 0.86, 0.90, 0.93, 0.89, 0.88, 0.97,|
Valid Loss: 0.350580, Acc: 0.82, 0.91, 0.96, 0.85, 0.89, 0.93, 0.90, 0.89, 0.98,|
 
S0/100 0.39 min|
Train Loss: 0.307363, Acc: 0.84, 0.91, 0.97, 0.88, 0.90, 0.93, 0.89, 0.90, 0.97,|
Valid Loss: 0.305221, Acc: 0.86, 0.91, 0.96, 0.85, 0.89, 0.93, 0.90, 0.90, 0.98,|
 
S0/150 0.58 min|
Train Loss: 0.284473, Acc: 0.86, 0.91, 0.97, 0.89, 0.90, 0.93, 0.91, 0.91, 0.97,|
Valid Loss: 0.282902, Acc: 0.86, 0.91, 0.96, 0.85, 0.89, 0.93, 0.91, 0.91, 0.98,|
 
S0/200 0.77 min|
Train Loss: 0.269990, Acc: 0.88, 0.91, 0.97, 0.89, 0.90, 0.93, 0.92, 0.92, 0.97,|
Valid Loss: 0.269347, Acc: 0.88, 0.91, 0.96, 0.85, 0.89, 0.93, 0.92, 0.91, 0.98,|
 
S0/250 0.96 min|
Train Loss: 0.259606, Acc: 0.89, 0.91, 0.97, 0.89, 0.90, 0.93, 0.92, 0.